In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
url = 'https://raw.githubusercontent.com/alura-cursos/Clusterizacao-dados-sem-rotulo/main/Dados/dados_mkt.csv'

# Coletando e ajustando dados

In [ ]:
df = pd.read_csv(url)
df

In [ ]:
df.info()

In [ ]:
df['sexo'].unique()

## Aplicando encoder

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(categories=[['F', 'M', 'NE']], sparse_output=False)

In [ ]:
encoded_sexo = encoder.fit_transform(df[['sexo']])
encoded_sexo

In [ ]:
encoded_df = pd.DataFrame(encoded_sexo,columns=encoder.get_feature_names_out(['sexo']))

In [ ]:
dados = pd.concat([df,encoded_df],axis=1).drop('sexo',axis=1)
dados

In [ ]:
import joblib

joblib.dump(encoder,'encoder.pkl')

## Desenvolvendo o modelo

In [ ]:
from sklearn.cluster import KMeans

mod_kmeans = KMeans(n_clusters=2, random_state=45)

In [ ]:
modelo = mod_kmeans.fit(dados)

# Avaliando o K-means

In [ ]:
mod_kmeans.inertia_

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_score(dados,mod_kmeans.predict(dados))

## Avaliando métricas para diferentes K

In [ ]:
def avaliacao(dados):
  inercia = []
  silhueta = []

  for k in range(2,21):
    kmeans = KMeans(n_clusters=k, random_state=45, n_init='auto')
    kmeans.fit(dados)
    inercia.append(kmeans.inertia_)
    silhueta.append(f'k={k} - '+ str(silhouette_score(dados, kmeans.predict(dados))))
  return silhueta, inercia

In [ ]:
silhueta, inercia = avaliacao(dados)
silhueta

## Aula 2.3 - Verificando a silhueta por amostras

In [ ]:
import matplotlib.cm as cm
from sklearn.metrics import silhouette_samples

def graf_silhueta (n_clusters, dados):

  # Aplica o KMeans ao conjunto de dados
  kmeans = KMeans(n_clusters=n_clusters, random_state=45, n_init = 'auto')
  cluster_previsoes = kmeans.fit_predict(dados)

  # Calcula o silhouette score médio
  silhueta_media = silhouette_score(dados, cluster_previsoes)
  print(f'Valor médio para {n_clusters} clusters: {silhueta_media:.3f}')

  # Calcula a pontuação de silhueta para cada amostra
  silhueta_amostra = silhouette_samples(dados, cluster_previsoes)

  # Configuração da figura para o gráfico de silhueta
  fig, ax1 = plt.subplots(1, 1)
  fig.set_size_inches(9, 7)

  # Limites do gráfico de silhueta
  ax1.set_xlim([-0.1, 1])
  ax1.set_ylim([0, len(dados) + (n_clusters + 1) * 10])

  y_lower = 10
  for i in range(n_clusters):
      ith_cluster_silhueta_amostra = silhueta_amostra[cluster_previsoes == i]
      ith_cluster_silhueta_amostra.sort()

      tamanho_cluster_i = ith_cluster_silhueta_amostra.shape[0]
      y_upper = y_lower + tamanho_cluster_i

      cor = cm.nipy_spectral(float(i) / n_clusters)
      ax1.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhueta_amostra,
                        facecolor=cor, edgecolor=cor, alpha=0.7)

      ax1.text(-0.05, y_lower + 0.5 * tamanho_cluster_i, str(i))
      y_lower = y_upper + 10  # 10 para o espaço entre gráficos

  # Linha vertical para a média do Silhouette Score
  ax1.axvline(x=silhueta_media, color='red', linestyle='--')

  ax1.set_title(f'Gráfico da Silhueta para {n_clusters} clusters')
  ax1.set_xlabel('Valores do coeficiente de silhueta')
  ax1.set_ylabel('Rótulo do cluster')

  ax1.set_yticks([])  # Remove os ticks do eixo y
  ax1.set_xticks([i/10.0 for i in range(-1, 11)])

  plt.show()

In [ ]:
graf_silhueta(2,dados)

### Outras visualizações

In [ ]:
def plot_cotovelo(inercia):
  plt.figure(figsize=(8,4))
  plt.plot(range(2,21),inercia,'bo-')
  plt.xlabel('Número de clusters')
  plt.ylabel('Inércia')
  plt.title('Método do Cotovelo para Determinação de k')
  plt.show()

In [ ]:
plot_cotovelo(inercia)

# Avaliação e ajuste de dados

In [ ]:
dados.describe()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

dados_escalados = scaler.fit_transform(dados)
dados_escalados

In [ ]:
dados_escalados = pd.DataFrame(dados_escalados, columns=dados.columns)
dados_escalados.describe()

In [ ]:
joblib.dump(scaler,'scaler.pkl')

## Verificando as métricas para os novos dados

In [ ]:
silhueta, inercia = avaliacao(dados_escalados)
silhueta

In [ ]:
graf_silhueta(3, dados_escalados)

In [ ]:
plot_cotovelo(inercia)

## Criando o melhor modelo

In [ ]:
modelo_kmeans = KMeans(n_clusters = 3, random_state = 45, n_init = 'auto')
modelo_kmeans.fit(dados_escalados)

In [ ]:
joblib.dump(modelo_kmeans, 'kmeans.pkl')

# Analisando as informações de cada cluster

In [ ]:
dados_analise = pd.DataFrame()

dados_analise[dados_escalados.columns] = scaler.inverse_transform(dados_escalados)

dados_analise

In [ ]:
dados_analise['cluster'] = modelo_kmeans.labels_

In [ ]:
cluster_media = dados_analise.groupby('cluster').mean()

cluster_media.T

In [ ]:
cluster_media = cluster_media.transpose()

cluster_media.columns = [0,1,2]

In [ ]:
cluster_media

## Descrevendo cada agrupamento

In [ ]:
cluster_media[0].sort_values(ascending=False)

In [ ]:
cluster_media[1].sort_values(ascending=False)

In [ ]:
cluster_media[2].sort_values(ascending=False)